In [27]:
import pandas as pd

# Load the Excel and CSV files
df_excel = pd.read_excel('31168- August 2024.xlsx')

df_csv = pd.read_excel('Delivery_Match_Check_from_Shopify_July_August.xlsx')



# Filter out rows where the relevant columns are empty
df_excel_filtered = df_excel.dropna(subset=['Date'])
df_csv_filtered = df_csv.dropna(subset=['Shipping'])


In [28]:

# Create an empty list to store matched rows
matched_data = []

# Iterate over rows in the CSV (df_csv_filtered)
for index_csv, row_csv in df_csv_filtered.iterrows():
    # Extract the 'Name' from df_csv
    csv_name = row_csv['Name']
    
    # Find matching names in df_excel (where the excel 'Name' contains the csv 'Name')
    matching_rows_excel = df_excel_filtered[df_excel_filtered['Name'].str.contains(csv_name, na=False)]
    
    # If a match is found, extract relevant columns and combine them
    for index_excel, row_excel in matching_rows_excel.iterrows():
        matched_row = {
            'Name (CSV)': row_csv['Name'],
            'Shipping': row_csv['Shipping'],
            'Fulfilled at': row_csv['Fulfilled at'],
            'Fulfillment Status': row_csv['Fulfillment Status'],
            'Date': row_excel['Date'],
            'Total Charge': row_excel['Total Charge'],
            'Shipping Charge' : row_excel['Domestic']

        }
        matched_data.append(matched_row)

# Convert the matched data to a new DataFrame
matched_df = pd.DataFrame(matched_data)

# Display the matched data
print("Matched Data:")
print(matched_df)

# Optionally, save the result to a new Excel or CSV file
# matched_df.to_excel('Matched_Shipping_Data.xlsx', index=False)
# matched_df.to_csv('Matched_Shipping_Data.csv', index=False)


Matched Data:
    Name (CSV)  Shipping Fulfilled at Fulfillment Status       Date  \
0    TRAU50379     10.45   2024-08-30          fulfilled 2024-08-30   
1    TRAU50378     17.57   2024-08-30          fulfilled 2024-08-30   
2    TRAU50377     61.38   2024-08-30          fulfilled 2024-08-30   
3    TRAU50376     23.72   2024-08-30          fulfilled 2024-08-30   
4    TRAU50375     15.26   2024-08-30          fulfilled 2024-08-30   
..         ...       ...          ...                ...        ...   
515  TRAU49482     18.08   2024-08-22          fulfilled 2024-08-22   
516  TRAU49480     18.08   2024-08-22          fulfilled 2024-08-22   
517  TRAU49399     28.65   2024-08-05          fulfilled 2024-08-05   
518  TRAU49387     35.38   2024-07-26          fulfilled 2024-08-05   
519  TRAU49380     99.74   2024-07-08          fulfilled 2024-08-05   

     Total Charge  Shipping Charge  
0           20.51            14.01  
1           39.07            30.62  
2           55.91     

In [29]:
matched_df['Coverage Status Total'] = matched_df.apply(
    lambda row: 'Undercoverage' if row['Shipping'] < row['Total Charge'] else 'Coverage',
    axis=1
)

matched_df['Coverage Status Shipping'] = matched_df.apply(
    lambda row: 'Undercoverage' if row['Shipping'] < row['Shipping Charge'] else 'Coverage',
    axis=1
)


In [30]:
matched_df

,Name (CSV),Shipping,Fulfilled at,Fulfillment Status,Date,Total Charge,Shipping Charge,Coverage Status Total,Coverage Status Shipping
0,TRAU50379,10.45,2024-08-30,fulfilled,2024-08-30,20.51,14.01,Undercoverage,Undercoverage
1,TRAU50378,17.57,2024-08-30,fulfilled,2024-08-30,39.07,30.62,Undercoverage,Undercoverage
2,TRAU50377,61.38,2024-08-30,fulfilled,2024-08-30,55.91,39.96,Coverage,Coverage
3,TRAU50376,23.72,2024-08-30,fulfilled,2024-08-30,51.29,37.14,Undercoverage,Undercoverage
4,TRAU50375,15.26,2024-08-30,fulfilled,2024-08-30,31.93,23.48,Undercoverage,Undercoverage
...,...,...,...,...,...,...,...,...,...
515,TRAU49482,18.08,2024-08-22,fulfilled,2024-08-22,27.41,18.96,Undercoverage,Undercoverage
516,TRAU49480,18.08,2024-08-22,fulfilled,2024-08-22,23.61,15.16,Undercoverage,Coverage
517,TRAU49399,28.65,2024-08-05,fulfilled,2024-08-05,50.80,33.80,Undercoverage,Undercoverage
518,TRAU49387,35.38,2024-07-26,fulfilled,2024-08-05,25.49,18.99,Coverage,Coverage


In [33]:
# Ensure the 'Shipping' and 'Shipping Charge' columns are numeric
matched_df['Shipping'] = pd.to_numeric(matched_df['Shipping'], errors='coerce')
matched_df['Shipping Charge'] = pd.to_numeric(matched_df['Shipping Charge'], errors='coerce')

# Calculate the difference between Shipping and Shipping Charge
matched_df['Shipping Difference'] = matched_df['Shipping'] - matched_df['Shipping Charge']

# Calculate the total difference
total_difference = matched_df['Shipping Difference'].sum()

# Calculate the total Shipping and total Shipping Charge
total_shipping = matched_df['Shipping'].sum()
total_shipping_charge = matched_df['Shipping Charge'].sum()

# Display the result
print("Matched Data with Shipping Difference:")
print(matched_df)

print(f"Total Shipping Charge: {total_shipping_charge}")
print(f"Total Shipping: {total_shipping}")
print(f"Total Difference between Shipping and Shipping Charge: {total_difference}")

# # Optionally, save the updated matched_df to a file
# matched_df.to_excel('Matched_Shipping_Data_With_Totals.xlsx', index=False)
# matched_df.to_csv('Matched_Shipping_Data_With_Totals.csv', index=False)


Matched Data with Shipping Difference:
    Name (CSV)  Shipping Fulfilled at Fulfillment Status       Date  \
0    TRAU50379     10.45   2024-08-30          fulfilled 2024-08-30   
1    TRAU50378     17.57   2024-08-30          fulfilled 2024-08-30   
2    TRAU50377     61.38   2024-08-30          fulfilled 2024-08-30   
3    TRAU50376     23.72   2024-08-30          fulfilled 2024-08-30   
4    TRAU50375     15.26   2024-08-30          fulfilled 2024-08-30   
..         ...       ...          ...                ...        ...   
515  TRAU49482     18.08   2024-08-22          fulfilled 2024-08-22   
516  TRAU49480     18.08   2024-08-22          fulfilled 2024-08-22   
517  TRAU49399     28.65   2024-08-05          fulfilled 2024-08-05   
518  TRAU49387     35.38   2024-07-26          fulfilled 2024-08-05   
519  TRAU49380     99.74   2024-07-08          fulfilled 2024-08-05   

     Total Charge  Shipping Charge Coverage Status Total  \
0           20.51            14.01         Under

In [34]:
# Save matched_df to an Excel file
matched_df.to_excel('Matched_Shipping_Data_With_Totals.xlsx', index=False)

# Warehouse 

In [2]:
# Load the Excel and CSV files
import pandas as pd
df_excel = pd.read_excel('31168- August 2024.xlsx')
df_excel

,Date,Order #,Name,Unnamed: 3,Cost Centre #,Cost Centre Name/Delivery Address,Unnamed: 6,Product,Description,Qty,Process Charge,Domestic,International,Total Charge,No. of Parcels,Weight,Cubic,Con. Number,Delivery Courier
0,NaN,1321868.0,TRAU49137/,NaN,DIG OPTIONS,DIG OPTIONS,NaN,TR80-SHELF4-BLK,SHELF KEYBOARD MONITOR,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-01 00:00:00,1321868.0,TRAU49137/,NaN,DIG OPTIONS,"Attn: NATHAN FERGUS ,UNIT 28 / 18 FORESHORE ST...",NaN,NaN,NaN,NaN,14.35,18.99,NaN,33.34,1.0,7.0,1.0,3303211321868,Direct Freight Express
2,NaN,1321908.0,DIG1049367/,NaN,DIG OPTIONS,DIG OPTIONS,NaN,TR-RSB4,UNIVERSAL SEAT BRACKETS FOR RECLINE SEATS AND ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-01 00:00:00,1321908.0,DIG1049367/,NaN,DIG OPTIONS,"Attn: NATHAN VA PAMELENN Phone: 0431373874, ,1...",NaN,NaN,NaN,NaN,6.50,14.02,NaN,20.52,1.0,5.0,1.0,M7U5162834,eParcel
4,NaN,1321909.0,DIG1049368/,NaN,DIG OPTIONS,DIG OPTIONS,NaN,TP-09S,Spare Part - TPMS Internal Sensor TP-09,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2422,NaN,1325874.0,TRAU50377-W6/168334,NaN,DIG OPTIONS,DIG OPTIONS,NaN,TR-GLOVE-M,BLACK GLOVES,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2423,2024-08-30 00:00:00,1325874.0,TRAU50377-W6/168334,NaN,DIG OPTIONS,"Attn: WAZEEN WAHAB WAZEEN WAHAB,14 CRIMSON DRI...",NaN,NaN,NaN,NaN,15.95,39.96,NaN,55.91,4.0,61.0,4.0,3303211325874,Direct Freight Express
2424,NaN,1325867.0,HIAU25114/168308,NaN,DIG OPTIONS,DIG OPTIONS,NaN,HI-038B,CAR SWITCH,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2425,2024-08-30 00:00:00,1325867.0,HIAU25114/168308,NaN,DIG OPTIONS,"Attn: STEPHEN DUNSTAN Phone: 0475195534, ,05, ...",NaN,NaN,NaN,NaN,7.25,11.31,NaN,18.56,1.0,0.3,1.0,M7U5164595,eParcel


: 